In [ ]:
import os
import sys

sys.path.append("../")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from natsort import natsorted

import torch
import random

from src.Metamodel.models.depthwiseNet import DepthNet
from src.Metamodel.data.dataloader import create_train_val_loader
from torch.utils.data import random_split, DataLoader

from sklearn.metrics import confusion_matrix, auc, roc_curve, roc_auc_score, f1_score, classification_report
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)
np.random.seed(0)
torch.cuda.manual_seed_all(0)

from src.FedMeta.data.data_loader import applewatch

In [ ]:
DATAPATH = "../data/new"
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = DepthNet(lengths=30, patch_size=30, in_chans=2, embed_dim=256, norm_layer=None, output_dim=3).to(device)

weightType = "macro" # macro weighted

In [ ]:
clients_results_meta = []

MODE = "meta" # "meta", "normal", "random"
MODELPATH = f"../src/FedMeta/model_file_cache/{MODE}"

In [ ]:
for i in range(10):
    PATH = os.path.join("../data/new", f"c{i+1}_data.csv")
    
    generator = torch.Generator()
    generator.manual_seed(0)
    
    train_dataset = applewatch(PATH=PATH, length=30)
    train_data, test_data = torch.utils.data.random_split(train_dataset,
                                                            [int(len(train_dataset)*0.8),
                                                            len(train_dataset)-int(len(train_dataset)*0.8)],
                                                            generator=generator)
    
    train_loader = torch.utils.data.DataLoader(
            train_data, batch_size=32, shuffle=True, drop_last=False
        )

    test_loader = torch.utils.data.DataLoader(
            test_data, batch_size=32, shuffle=True, drop_last=False
        )
        
    labels2 = []

    for data in test_loader:
        inputs, label = data[0], data[1]

        labels2.extend(label.numpy())
    labels2 = np.array(labels2)
    print("##########################")
    print(len(labels2[labels2 == 2]))
    print("##########################")

In [ ]:
train_loader = torch.utils.data.DataLoader(
            train_data, batch_size=32, shuffle=True, drop_last=True
        )

test_loader = torch.utils.data.DataLoader(
            test_data, batch_size=32, shuffle=True, drop_last=True
        )

In [ ]:
labels = []

for data in train_loader:
    print(data[0].shape)
    print(data[1].shape)
    inputs, label = data[0], data[1]

    labels.extend(label.numpy())

In [ ]:
labels2 = []

for data in test_loader:
    inputs, input = data[0], data[1]

    labels2.extend(label.numpy())

In [ ]:
len(labels2)

In [ ]:
len(labels)